In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import shutil

In [27]:
# Data directory
data_dir = '../Data_Q2/train_data'

folders = ['Cloudy', 'Foggy', 'Rainy', 'Snowy', 'Sunny']

for folder_name in folders:
    folder_path = os.path.join(data_dir, folder_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

for filename in os.listdir(data_dir):
    if os.path.isfile(os.path.join(data_dir, filename)):  
        for folder_name in folders:
            if folder_name in filename:
                source_path = os.path.join(data_dir, filename)
                destination_path = os.path.join(data_dir, folder_name, filename)
                shutil.move(source_path, destination_path)

In [28]:
# Data directory
data_dir = '../Data_Q2/train_data'

# Image size and batch size settings
image_size = (224, 224)
batch_size = 32

# Initialize ImageDataGenerator with rescaling
datagen = ImageDataGenerator(rescale=1./255)

# Load images from directory
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')


Found 250 images belonging to 5 classes.


In [29]:
def build_vgg16(input_shape=(224, 224, 3), num_classes=5):
    # Define the input
    input_tensor = Input(shape=input_shape)

    # First block
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_tensor)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Second block
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Third block
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Fourth block
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Fifth block
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Flatten and Dense layers
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=x)
    return model

In [30]:
# Build the model
model = build_vgg16()

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_13 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_14 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 112, 112, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_16 (Conv2D)          (None, 112, 112, 128)     147584    
                                                           

In [ ]:
# Train the model
history = model.fit(train_generator, epochs=50)

Epoch 1/50
1/8 [==>...........................] - ETA: 51s - loss: 1.6101 - accuracy: 0.1562

In [ ]:
# Evaluate the model on the training set
train_loss, train_accuracy = model.evaluate(train_generator)
print("Accuracy on training set:", train_accuracy)

In [ ]:
import matplotlib.pyplot as plt

# Plot the training accuracy and loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.title('Training Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss')
plt.title('Training Loss')
plt.legend()
plt.show()